In [83]:
import pandas as pd
from sklearn.datasets import fetch_openml

In [84]:
dataset=fetch_openml('titanic',version=1,as_frame=True)
df=pd.DataFrame(dataset.data)

In [85]:
df.head(3)

,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [86]:
df['servived']=dataset.target
df.head(3)

,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest,servived
0,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO",1
1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON",1
2,1,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON",0


In [87]:
df.isnull().sum()

pclass          0
name            0
sex             0
age           263
sibsp           0
parch           0
ticket          0
fare            1
cabin        1014
embarked        2
boat          823
body         1188
home.dest     564
servived        0
dtype: int64

In [88]:
df.drop(['name','ticket','cabin','boat','body','home.dest'],axis=1,inplace=True)

In [89]:
from sklearn.impute import SimpleImputer
mean=SimpleImputer(strategy='mean')
df['age']=mean.fit_transform(df[['age']])
df['fare']=mean.fit_transform(df[['fare']])
mode=SimpleImputer(strategy='most_frequent')
df['embarked']=mode.fit_transform(df[['embarked']]).ravel()

In [90]:
df.isnull().sum()

pclass      0
sex         0
age         0
sibsp       0
parch       0
fare        0
embarked    0
servived    0
dtype: int64

In [91]:
df['family']=df['sibsp']+df['parch']
df.loc[df['family']>0,'travelled_alone']=0
df.loc[df['family']==0,'travelled_alone']=1
df.drop(['sibsp','parch','family'],axis=1,inplace=True)

In [92]:
df.head(3)

,pclass,sex,age,fare,embarked,servived,travelled_alone
0,1,female,29.0000,211.3375,S,1,1.0
1,1,male,0.9167,151.5500,S,1,0.0
2,1,female,2.0000,151.5500,S,0,0.0


In [93]:
df=pd.get_dummies(df,columns=['sex','embarked'],drop_first=True)
df.head(3)

,pclass,age,fare,servived,travelled_alone,sex_male,embarked_Q,embarked_S
0,1,29.0000,211.3375,1,1.0,False,False,True
1,1,0.9167,151.5500,1,0.0,True,False,True
2,1,2.0000,151.5500,0,0.0,False,False,True


In [94]:
x=df.drop('servived',axis=1)

In [95]:
y=df['servived']

In [96]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [97]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [98]:
model=LogisticRegression()
model.fit(x_train,y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [102]:
y_test_pred=model.predict(x_test)

In [103]:
print("\nClassification Report:\n", classification_report(y_test, y_test_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_test_pred))


Classification Report:
               precision    recall  f1-score   support

           0       0.75      0.89      0.82       144
           1       0.83      0.64      0.72       118

    accuracy                           0.78       262
   macro avg       0.79      0.77      0.77       262
weighted avg       0.79      0.78      0.77       262


Confusion Matrix:
 [[128  16]
 [ 42  76]]


In [104]:
print("Test Accuracy:", accuracy_score(y_test, y_test_pred))

Test Accuracy: 0.7786259541984732
